In [48]:
# Run this cell once to install everything
!pip install -q numpy scipy soundfile librosa samplerate pydub
!pip install "audio-separator[gpu]"  # or "audio-separator" for the CPU version
!apt update && apt install -y ffmpeg


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease   
Get:5 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1268 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease             
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4976 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3159 k

In [51]:
#
# Cell 2: Define the Pipeline (Path Corrected)
#
import os
import sys
import traceback
import logging
import shutil
import numpy as np
import soundfile as sf
import samplerate
from pydub import AudioSegment, effects
from audio_separator.separator import Separator

class AudioProcessingPipeline:
    """
    A pipeline to process audio files, using a temporary directory for
    intermediate files and saving only the final output.
    """

    def __init__(self, input_dir, output_dir, model_name="UVR-MDX-NET-Inst_HQ_3.onnx", target_sr=16000):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.target_sr = target_sr
        
        self.temp_dir = os.path.join(self.output_dir, 'temp_audio')

        os.makedirs(self.input_dir, exist_ok=True)
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.temp_dir, exist_ok=True)

        print("Initializing audio separator...")
        self.separator = Separator(
            log_level=logging.INFO,
            output_dir=self.temp_dir 
        )
        self.separator.load_model(model_filename=model_name)
        print("Model loaded successfully.")

    def _convert_to_wav(self, file_path):
        """Converts audio to a WAV file inside the temp directory."""
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        wav_path = os.path.join(self.temp_dir, f"{base_name}_initial.wav")
        
        print(f"Converting {file_path} to temporary WAV...")
        try:
            audio = AudioSegment.from_file(file_path)
            audio.export(wav_path, format='wav')
            print(f"  - Temporary WAV saved: {wav_path}")
            return wav_path
        except Exception as e:
            print(f"Error converting {file_path}: {e}")
            return None

    def _separate_vocals(self, audio_path):
        """Separates vocals, outputting stems to the temp directory."""
        print(f"Separating vocals for {os.path.basename(audio_path)}...")
        try:
            # This returns a list of just the filenames, not full paths
            output_filenames = self.separator.separate(audio_path)
            vocal_filename = None
            for fname in output_filenames:
                if '(Vocals)' in fname:
                    vocal_filename = fname
                    break
            
            if vocal_filename:
                # --- THIS IS THE FIX ---
                # Manually construct the full path to the output file
                full_vocal_path = os.path.join(self.temp_dir, vocal_filename)
                # --- END FIX ---
                print(f"  - Vocal stem found. Full path: {full_vocal_path}")
                return full_vocal_path
            else:
                print("Error: Vocal stem filename not found in separation output.")
                return None
        except Exception as e:
            print(f"Error during vocal separation for {audio_path}: {e}")
            traceback.print_exc()
            return None

    def _post_process_vocals(self, vocal_path, original_filename):
        """Processes the vocal stem and saves the final file to the main output directory."""
        print(f"Post-processing vocal stem: {vocal_path}")
        try:
            # Now, vocal_path is the full, correct path
            audio, sr = sf.read(vocal_path)
            print(f"  - Original sample rate: {sr} Hz")

            ratio = self.target_sr / sr
            audio_resampled = samplerate.resample(audio, ratio, 'sinc_best')
            
            temp_resampled_path = os.path.join(self.temp_dir, "temp_resampled.wav")
            sf.write(temp_resampled_path, audio_resampled.astype(np.float32), self.target_sr)
            print(f"  - Resampled to {self.target_sr}Hz.")

            audio_pd = AudioSegment.from_file(temp_resampled_path)
            normalized_audio = effects.normalize(audio_pd)
            print("  - Audio normalized.")

            final_audio = normalized_audio.set_frame_rate(self.target_sr).set_channels(1).set_sample_width(2)
            
            base_name = os.path.splitext(original_filename)[0]
            final_filename = f"{base_name}_denoised.wav"
            final_path = os.path.join(self.output_dir, final_filename)
            final_audio.export(final_path, format='wav')
            print(f"  - ✅ Final processed file saved to: {final_path}")

            return final_path

        except Exception as e:
            print(f"Error during post-processing for {vocal_path}: {e}")
            traceback.print_exc()
            return None

    def _cleanup(self):
        """Removes the temporary directory and its contents."""
        print("\nCleaning up temporary files...")
        try:
            shutil.rmtree(self.temp_dir)
            print(f"  - Successfully removed temp folder: {self.temp_dir}")
        except Exception as e:
            print(f"Error during cleanup: {e}")

    def run(self):
        """Executes the full pipeline and cleans up afterwards."""
        print(f"\nStarting audio processing pipeline...")
        print(f"Input directory: '{self.input_dir}'")
        print(f"Output directory: '{self.output_dir}'")
        print(f"Temporary directory: '{self.temp_dir}'\n")

        supported_formats = ('.mp3', '.wav', '.flac', '.m4a')
        audio_files = [f for f in os.listdir(self.input_dir) if f.lower().endswith(supported_formats)]

        if not audio_files:
            print("No audio files found in the input directory. Exiting.")
            self._cleanup()
            return

        for filename in audio_files:
            try:
                print(f"--- Processing file: {filename} ---")
                file_path = os.path.join(self.input_dir, filename)
                
                wav_path = self._convert_to_wav(file_path)
                if not wav_path:
                    continue

                vocal_stem_path = self._separate_vocals(wav_path)
                if not vocal_stem_path:
                    continue

                self._post_process_vocals(vocal_stem_path, filename)
                print(f"--- Finished processing {filename} ---\n")

            except Exception as e:
                print(f"An unexpected error occurred while processing {filename}: {e}")
                traceback.print_exc()
                continue
        
        self._cleanup()
        print("Pipeline finished.")


In [52]:
# Paste this into the third cell to run the process

# Define the input and output directories
INPUT_FOLDER = 'input_audio'
OUTPUT_FOLDER = 'output_audio'

# --- SETUP FOR DEMONSTRATION ---
print("Setting up demo environment...")
os.makedirs(INPUT_FOLDER, exist_ok=True)

# Create a dummy silent MP3 file for testing if no audio files are present.
# In your use case, place your actual audio files in the 'input_audio' folder.
if not any(f.lower().endswith(('.mp3', '.wav', '.flac', '.m4a')) for f in os.listdir(INPUT_FOLDER)):
    dummy_file_path = os.path.join(INPUT_FOLDER, 'sample_audio.mp3')
    print(f"Creating a dummy audio file for demonstration: {dummy_file_path}")
    try:
        silent_segment = AudioSegment.silent(duration=10000) 
        silent_segment.export(dummy_file_path, format='mp3')
    except Exception as e:
        print("\n---")
        print("Could not create a dummy MP3 file. This might be due to a missing ffmpeg/ffprobe executable.")
        print("Please ensure ffmpeg is installed and accessible in your system's PATH.")
        print(f"Error: {e}")
        print("---\n")
# --- END OF DEMO SETUP ---

# Initialize and run the pipeline
# This line calls the corrected code from Cell 2
pipeline = AudioProcessingPipeline(input_dir=INPUT_FOLDER, output_dir=OUTPUT_FOLDER)
pipeline.run()

2025-07-22 17:10:15,910 - INFO - separator - Separator version 0.35.0 instantiating with output_dir: output_audio/temp_audio, output_format: WAV
2025-07-22 17:10:15,911 - INFO - separator - Using model directory from model_file_dir parameter: /tmp/audio-separator-models/
2025-07-22 17:10:15,912 - INFO - separator - Operating System: Linux #126-Ubuntu SMP Mon Jul 1 10:14:24 UTC 2024
2025-07-22 17:10:15,913 - INFO - separator - System: Linux Node: audiodenoising-yvnrkzo7g9t97faw-0 Release: 5.15.0-116-generic Machine: x86_64 Proc: x86_64
2025-07-22 17:10:15,914 - INFO - separator - Python Version: 3.11.11
2025-07-22 17:10:15,914 - INFO - separator - PyTorch Version: 2.7.1+cu126
2025-07-22 17:10:15,965 - INFO - separator - FFmpeg installed: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
2025-07-22 17:10:15,969 - INFO - separator - ONNX Runtime GPU package installed with version: 1.22.0
2025-07-22 17:10:15,970 - INFO - separator - ONNX Runtime CPU packag

Setting up demo environment...
Initializing audio separator...


2025-07-22 17:10:16,358 - INFO - separator - Load model duration: 00:00:00


Model loaded successfully.

Starting audio processing pipeline...
Input directory: 'input_audio'
Output directory: 'output_audio'
Temporary directory: 'output_audio/temp_audio'

--- Processing file: 20241115-165038_08910460989-all.mp3 ---
Converting input_audio/20241115-165038_08910460989-all.mp3 to temporary WAV...


2025-07-22 17:10:16,664 - INFO - separator - Processing file: output_audio/temp_audio/20241115-165038_08910460989-all_initial.wav
2025-07-22 17:10:16,665 - INFO - separator - Starting separation process for audio_file_path: output_audio/temp_audio/20241115-165038_08910460989-all_initial.wav


  - Temporary WAV saved: output_audio/temp_audio/20241115-165038_08910460989-all_initial.wav
Separating vocals for 20241115-165038_08910460989-all_initial.wav...


100%|██████████| 77/77 [00:08<00:00,  9.00it/s]
2025-07-22 17:10:46,630 - INFO - mdx_separator - Saving Vocals stem to 20241115-165038_08910460989-all_initial_(Vocals)_UVR-MDX-NET-Inst_HQ_3.wav...
2025-07-22 17:10:46,674 - INFO - common_separator - Audio duration is 0.12 hours (442.30 seconds).
2025-07-22 17:10:46,674 - INFO - common_separator - Using pydub for writing.
2025-07-22 17:10:47,040 - INFO - mdx_separator - Saving Instrumental stem to 20241115-165038_08910460989-all_initial_(Instrumental)_UVR-MDX-NET-Inst_HQ_3.wav...
2025-07-22 17:10:47,066 - INFO - common_separator - Audio duration is 0.12 hours (442.30 seconds).
2025-07-22 17:10:47,067 - INFO - common_separator - Using pydub for writing.
2025-07-22 17:10:47,687 - INFO - common_separator - Clearing input audio file paths, sources and stems...
2025-07-22 17:10:47,689 - INFO - separator - Separation duration: 00:00:31


  - Vocal stem found. Full path: output_audio/temp_audio/20241115-165038_08910460989-all_initial_(Vocals)_UVR-MDX-NET-Inst_HQ_3.wav
Post-processing vocal stem: output_audio/temp_audio/20241115-165038_08910460989-all_initial_(Vocals)_UVR-MDX-NET-Inst_HQ_3.wav
  - Original sample rate: 44100 Hz
  - Resampled to 16000Hz.
  - Audio normalized.
  - ✅ Final processed file saved to: output_audio/20241115-165038_08910460989-all_denoised.wav
--- Finished processing 20241115-165038_08910460989-all.mp3 ---


Cleaning up temporary files...
  - Successfully removed temp folder: output_audio/temp_audio
Pipeline finished.
